For EDA please visit https://www.kaggle.com/rohitsingh9990/m5-forecasting-eda-feature-engineering

version3: LB: 0.60869

version5: LB: ??

In [1]:

import pandas as pd
import numpy as np
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn import preprocessing, metrics
import gc
import joblib
import warnings
warnings.filterwarnings('ignore')

## 1. Loading Data

In [2]:
INPUT_DIR_PATH = 'data/'

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


def read_data():
    sell_prices_df = pd.read_csv(INPUT_DIR_PATH + 'sell_prices.csv')
    sell_prices_df = reduce_mem_usage(sell_prices_df)
    print('Sell prices has {} rows and {} columns'.format(sell_prices_df.shape[0], sell_prices_df.shape[1]))

    calendar_df = pd.read_csv(INPUT_DIR_PATH + 'calendar.csv')
    calendar_df = reduce_mem_usage(calendar_df)
    print('Calendar has {} rows and {} columns'.format(calendar_df.shape[0], calendar_df.shape[1]))

    sales_train_validation_df = pd.read_csv(INPUT_DIR_PATH + 'sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation_df.shape[0], sales_train_validation_df.shape[1]))

    submission_df = pd.read_csv(INPUT_DIR_PATH + 'sample_submission.csv')
    return sell_prices_df, calendar_df, sales_train_validation_df, submission_df
    

In [4]:
sell_prices_df, calendar_df, sales_train_validation_df, submission_df = read_data()

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Sales train validation has 30490 rows and 1919 columns


In [5]:
sell_prices_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [6]:
prices_item = sell_prices_df[sell_prices_df['item_id']==0]
#print(prices_item)
prices_item[prices_item['store_id']==0]['sell_price']

Series([], Name: sell_price, dtype: float16)

In [7]:
calendar_df = calendar_df.fillna('Nothing')

In [8]:
# np.unique(calendar_df['event_name_1'].values)

In [9]:
# np.unique(calendar_df['event_type_1'].values)

In [10]:
# np.unique(calendar_df['snap_CA'].values)

In [11]:
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,Nothing,Nothing,Nothing,Nothing,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,Nothing,Nothing,Nothing,Nothing,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,Nothing,Nothing,Nothing,Nothing,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,Nothing,Nothing,Nothing,Nothing,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,Nothing,Nothing,Nothing,Nothing,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,Nothing,Nothing,Nothing,Nothing,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,Nothing,Nothing,Nothing,Nothing,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,Nothing,Nothing,Nothing,Nothing,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,Nothing,Nothing,Nothing,Nothing,0,0,0


In [12]:
calendar_df['date']

0       2011-01-29
1       2011-01-30
2       2011-01-31
3       2011-02-01
4       2011-02-02
           ...    
1964    2016-06-15
1965    2016-06-16
1966    2016-06-17
1967    2016-06-18
1968    2016-06-19
Name: date, Length: 1969, dtype: object

In [13]:
calendar_df['date'] = pd.to_datetime(calendar_df['date'], format='%Y-%m-%d', errors='coerce')

In [14]:
calendar_df['date']

0      2011-01-29
1      2011-01-30
2      2011-01-31
3      2011-02-01
4      2011-02-02
          ...    
1964   2016-06-15
1965   2016-06-16
1966   2016-06-17
1967   2016-06-18
1968   2016-06-19
Name: date, Length: 1969, dtype: datetime64[ns]

In [15]:
sales_train_validation_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [16]:
values = sales_train_validation_df['store_id'].values
print(np.unique(values), np.unique(values).shape[0])

['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3'] 10


In [17]:
submission_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
NUM_ITEMS = sales_train_validation_df.shape[0]  # 30490
DAYS_PRED = 28
#nrows = 365 * 2 * NUM_ITEMS

In [19]:
def encode_categorical(df, cols):
    for col in cols:
        # Leave NaN as it is.
        le = preprocessing.LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index).astype('category')

    return df


calendar_df = encode_categorical(calendar_df, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]).pipe(reduce_mem_usage)
sales_train_validation_df = encode_categorical(sales_train_validation_df, ["item_id", "dept_id", "cat_id", "store_id", "state_id"]).pipe(reduce_mem_usage)
sell_prices_df = encode_categorical(sell_prices_df, ["item_id", "store_id"]).pipe(reduce_mem_usage)

Mem. usage decreased to  0.07 Mb (0.0% reduction)
Mem. usage decreased to 94.11 Mb (78.9% reduction)
Mem. usage decreased to 45.77 Mb (0.0% reduction)


In [20]:
calendar_df["event_name_1"]

0       19
1       19
2       19
3       19
4       19
        ..
1964    19
1965    19
1966    19
1967    19
1968    16
Name: event_name_1, Length: 1969, dtype: category
Categories (31, int64): [0, 1, 2, 3, ..., 27, 28, 29, 30]

In [21]:
sales_train_validation_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1,0,1,4,0,0,5,0,0,0,0,0,0,2,

In [22]:
# res = pd.melt(sales_train_validation_df, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
# res

In [23]:
# test1_rows = [row for row in submission_df['id'] if 'validation' in row]
# test2_rows = [row for row in submission_df['id'] if 'evaluation' in row]
# test1 = submission_df[submission_df['id'].isin(test1_rows)]
# test2 = submission_df[submission_df['id'].isin(test2_rows)]

# # change column names
# test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
#                   'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
# test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
#                   'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']

# # get product table
# product = sales_train_validation_df[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

# # merge with product table
# test2['id'] = test2['id'].str.replace('_evaluation','_validation')
# test1 = test1.merge(product, how = 'left', on = 'id')
# test2 = test2.merge(product, how = 'left', on = 'id')
# test2['id'] = test2['id'].str.replace('_validation','_evaluation')
# test2

## 2. Merging dataframes

In [24]:
# function to read the data and merge it

def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    sales_train_validation = sales_train_validation#.iloc[-nrows:,:]
    
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'

    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    
    valid_days = [f'd_{i}' for i in range(1000, 1970)]
    data = data[data['day'].isin(valid_days)]
    
    # get only a sample for fst training
#     data = data.loc[nrows:]
    
    # drop some calendar features
    calendar.drop(['weekday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'test2']

    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'],)
        data.drop(['d', 'day'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    gc.collect()
    
    return data

In [25]:
# nrows = 365 * 2 * NUM_ITEMS

nrows = 27500000
data = melt_and_merge(calendar_df, sell_prices_df, sales_train_validation_df, submission_df, nrows = nrows, merge = True)
# nrows = 27500000
data = reduce_mem_usage(data)

Melted sales train validation has 58327370 rows and 8 columns
Mem. usage decreased to 1335.11 Mb (0.0% reduction)
Our final dataset to train has 28721580 rows and 19 columns
Mem. usage decreased to 1424.44 Mb (10.3% reduction)


In [26]:
data

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,wday,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2013-10-24,11339,6,19,2,3,1,0,0,0,8.257812
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0,train,2013-10-24,11339,6,19,2,3,1,0,0,0,3.970703
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,0,train,2013-10-24,11339,6,19,2,3,1,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1,train,2013-10-24,11339,6,19,2,3,1,0,0,0,4.640625
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,1,train,2013-10-24,11339,6,19,2,3,1,0,0,0,3.080078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28721575,FOODS_3_823_WI_3_validation,1432,2,0,9,2,0,test1,2016-05-22,11617,2,19,2,3,1,0,0,0,2.980469
28721576,FOODS_3_824_WI_3_validation,1433,2,0,9,2,0,test1,2016-05-22,11617,2,19,2,3,1,0,0,0,2.480469
28721577,FOODS_3_825_WI_3_validation,1434,2,0,9,2,0,test1,2016-05-22,11617,2,19,2,3,1,0,0,0,3.980469
28721578,FOODS_3_826_WI_3_validation,1435,2,0,9,2,0,test1,2016-05-22,11617,2,19,2,3,1,0,0,0,1.280273


In [27]:
full_demand_data = data.set_index(
    ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "date"])[["demand"]].unstack(
        level=-1)
full_demand_data

demand  \
date                                                                     2013-10-24   
id                              item_id dept_id cat_id store_id state_id              
FOODS_1_001_CA_1_validation     0       0       0      0        0                 1   
FOODS_1_001_CA_2_validation     0       0       0      1        0                 0   
FOODS_1_001_CA_3_validation     0       0       0      2        0                 0   
FOODS_1_001_CA_4_validation     0       0       0      3        0                 0   
FOODS_1_001_TX_1_validation     0       0       0      4        1                 0   
...                                                                             ...   
HOUSEHOLD_2_516_TX_2_validation 3048    6       2      5        1                 0   
HOUSEHOLD_2_516_TX_3_validation 3048    6       2      6        1                 0   
HOUSEHOLD_2_516_WI_1_validation 3048    6       2      7        2                 0   
HOUSEHOLD_2_516_WI_2_validation 3048    6       2      8        2                 0   
HOUSEHOLD_2_516_WI_3_validation 3048    6       2      9        2                 0   

                                                                                     \
date                                                                     2013-10-25   
id                              item_id dept_id cat_id store_id state_id              
FOODS_1_001_CA_1_validation     0       0       0      0        0                 0   
FOODS_1_001_CA_2_validation     0       0       0      1        0                 6   
FOODS_1_001_CA_3_validation     0       0       0      2        0                 1   
FOODS_1_001_CA_4_validation     0       0       0      3        0                 0   
FOODS_1_001_TX_1_validation     0       0       0      4        1                 1   
...                                                                             ...   
HOUSEHOLD_2_516_TX_2_validation 3048    6       2      5        1                 0   
HOUSEHOLD_2_516_TX_3_validation 3048    6       2      6        1                 0   
HOUSEHOLD_2_516_WI_1_validation 3048    6       2      7        2                 0   
HOUSEHOLD_2_516_WI_2_validation 3048    6       2      8        2                 0   
HOUSEHOLD_2_516_WI_3_validation 3048    6       2      9        2                 0   

                                                                                     \
date                                                                     2013-10-26   
id                              item_id dept_id cat_id store_id state_id              
FOODS_1_001_CA_1_validation     0       0       0      0        0                 1   
FOODS_1_001_CA_2_validation     0       0       0      1        0                 1   
FOODS_1_001_CA_3_validation     0       0       0      2        0                 3   
FOODS_1_001_CA_4_validation     0       0       0      3        0                 0   
FOODS_1_001_TX_1_validation     0       0       0      4        1                 1   
...                                                                             ...   
HOUSEHOLD_2_516_TX_2_validation 3048    6       2      5        1                 2   
HOUSEHOLD_2_516_TX_3_validation 3048    6       2      6        1                 1   
HOUSEHOLD_2_516_WI_1_validation 3048    6       2      7        2                 0   
HOUSEHOLD_2_516_WI_2_validation 3048    6       2      8        2                 0   
HOUSEHOLD_2_516_WI_3_validation 3048    6       2      9        2                 1   

                                                                                     \
date                                                                     2013-10-27   
id                              item_id dept_id cat_id store_id state_id              
FOODS_1_001_CA_1_validation     0       0       0      0        0                 0   
FOODS_1_001_CA_2_validation     0       0       0      1        0       

In [28]:
full_price_data = data.set_index(
     ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "date"])[["sell_price"]].unstack(
        level=-1)
full_price_data

sell_price  \
date                                                                     2013-10-24   
id                              item_id dept_id cat_id store_id state_id              
FOODS_1_001_CA_1_validation     0       0       0      0        0          2.240234   
FOODS_1_001_CA_2_validation     0       0       0      1        0          2.240234   
FOODS_1_001_CA_3_validation     0       0       0      2        0          2.240234   
FOODS_1_001_CA_4_validation     0       0       0      3        0          2.240234   
FOODS_1_001_TX_1_validation     0       0       0      4        1          2.240234   
...                                                                             ...   
HOUSEHOLD_2_516_TX_2_validation 3048    6       2      5        1          5.941406   
HOUSEHOLD_2_516_TX_3_validation 3048    6       2      6        1          5.941406   
HOUSEHOLD_2_516_WI_1_validation 3048    6       2      7        2          5.941406   
HOUSEHOLD_2_516_WI_2_validation 3048    6       2      8        2          5.941406   
HOUSEHOLD_2_516_WI_3_validation 3048    6       2      9        2          5.941406   

                                                                                     \
date                                                                     2013-10-25   
id                              item_id dept_id cat_id store_id state_id              
FOODS_1_001_CA_1_validation     0       0       0      0        0          2.240234   
FOODS_1_001_CA_2_validation     0       0       0      1        0          2.240234   
FOODS_1_001_CA_3_validation     0       0       0      2        0          2.240234   
FOODS_1_001_CA_4_validation     0       0       0      3        0          2.240234   
FOODS_1_001_TX_1_validation     0       0       0      4        1          2.240234   
...                                                                             ...   
HOUSEHOLD_2_516_TX_2_validation 3048    6       2      5        1          5.941406   
HOUSEHOLD_2_516_TX_3_validation 3048    6       2      6        1          5.941406   
HOUSEHOLD_2_516_WI_1_validation 3048    6       2      7        2          5.941406   
HOUSEHOLD_2_516_WI_2_validation 3048    6       2      8        2          5.941406   
HOUSEHOLD_2_516_WI_3_validation 3048    6       2      9        2          5.941406   

                                                                                     \
date                                                                     2013-10-26   
id                              item_id dept_id cat_id store_id state_id              
FOODS_1_001_CA_1_validation     0       0       0      0        0          2.240234   
FOODS_1_001_CA_2_validation     0       0       0      1        0          2.240234   
FOODS_1_001_CA_3_validation     0       0       0      2        0          2.240234   
FOODS_1_001_CA_4_validation     0       0       0      3        0          2.240234   
FOODS_1_001_TX_1_validation     0       0       0      4        1          2.240234   
...                                                                             ...   
HOUSEHOLD_2_516_TX_2_validation 3048    6       2      5        1          5.941406   
HOUSEHOLD_2_516_TX_3_validation 3048    6       2      6        1          5.941406   
HOUSEHOLD_2_516_WI_1_validation 3048    6       2      7        2          5.941406   
HOUSEHOLD_2_516_WI_2_validation 3048    6       2      8        2          5.941406   
HOUSEHOLD_2_516_WI_3_validation 3048    6       2      9        2          5.941406   

                                                                                     \
date                                                                     2013-10-27   
id                              item_id dept_id cat_id store_id state_id              
FOODS_1_001_CA_1_validation     0       0       0      0        0          2.240234   
FOODS_1_001_CA_2_validation     0       0       0      1        0   

In [29]:
def get_timespan(df, dt, minus, periods, freq='D'):
    #print(pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq))
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [30]:
from datetime import date, timedelta
def prepare_dataset(demand_data, price_data, time_edge, calendar_data, is_train=False, name_prefix=None):
    date_now = str(calendar_data['date'].values[time_edge]).split('T')[0]
    year, month, day = np.array(date_now.split('-')).astype(np.int)
    date_now = date(year, month, day)
    print(date_now)
    X = {}
    prices_after = get_timespan(price_data, date_now + timedelta(days=28), 56, 8, '7D').values
    prices_year_max = get_timespan(price_data, date_now, 30*7, 30, '7D').values.max(axis=1)
    prices_year_min = get_timespan(price_data, date_now, 30*7, 30, '7D').values.min(axis=1)
    
    
    for i in range(8):
        X[f'price_after_week_{i}'] = (prices_after[:, i] - prices_year_min) / (1 + prices_year_max - prices_year_min)
    for i in range(1, 8):
        X[f'price_diff_after_week_{i}'] = prices_after[:, i] - prices_after[:, i-1]
        
    for i in [3, 7, 14, 30, 60, 140]:
        values = get_timespan(demand_data, date_now, i, i, 'D')
        X['demand_diff_%s_mean' % i] = values.diff(axis=1).mean(axis=1).values
        X['demand_mean_%s_decay' % i] = (values * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['demand_mean_%s' % i] = values.mean(axis=1).values
        X['demand_median_%s' % i] = values.median(axis=1).values
        X['demand_min_%s' % i] = values.min(axis=1).values
        X['demand_max_%s' % i] = values.max(axis=1).values
        X['demand_std_%s' % i] = values.std(axis=1).values
        
    for i in [3, 7, 14]:
        values = get_timespan(demand_data, date_now, i*7, i, '7D')
        X['demand_thiswd_diff_%s_mean' % i] = values.diff(axis=1).mean(axis=1).values
        X['demand_thiswd_mean_%s_decay' % i] = (values * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['demand_thiswd_mean_%s' % i] = values.mean(axis=1).values
        X['demand_thiswd_median_%s' % i] = values.median(axis=1).values
        X['demand_thiswd_min_%s' % i] = values.min(axis=1).values
        X['demand_thiswd_max_%s' % i] = values.max(axis=1).values
        X['demand_thiswd_std_%s' % i] = values.std(axis=1).values
    
    for i in [4, 10]:
        values = get_timespan(price_data, date_now, i*7, i, '7D')
        X['price_diff_%s_mean' % i] = values.diff(axis=1).mean(axis=1).values
        X['price_mean_%s_decay' % i] = (values * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['price_mean_%s' % i] = values.mean(axis=1).values
        X['price_min_%s' % i] = values.min(axis=1).values
        X['price_max_%s' % i] = values.max(axis=1).values
        X['price_std_%s' % i] = values.std(axis=1).values
    
    prices_150 = get_timespan(price_data, date_now, 150, 150, 'D').values
    demand_150 = get_timespan(demand_data, date_now, 150, 150, 'D').values
    prices_150[np.isnan(prices_150)] = 0.5
    
    weights = np.sum(prices_150 * demand_150, axis=1)

    X['weights'] = weights
    
    for i in range(1, 16):
        X['day_shift_%s' % i] = get_timespan(demand_data, date_now, i, 1, 'D').mean(axis=1).values
    
    for i in range(1, 15):
        X['day_diff_shift_%s' % i] = get_timespan(demand_data, date_now, i, 1, 'D').mean(axis=1).values - \
                                     get_timespan(demand_data, date_now, i+1, 1, 'D').mean(axis=1).values
    
    
#     X['dayofweek'] = date_now.weekday()
#     X["is_weekend"] = (date_now.weekday() in [5,6])
#     for key in X:
#         print(key, X[key].shape)
    X = pd.DataFrame(X)
    
    #cal_features = ['event_type_1', 'event_name_2', 'event_type_2', 'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI']
    
#     for cal_f in cal_features:
#         for i in np.arange(-3, 28):
#             X[f'{cal_f}_after_{i}'] = calendar_data[cal_f].values[time_edge + i]
#             X[f'{cal_f}_after_{i}'] = X[f'{cal_f}_after_{i}'].astype('category')
    X['time_edge'] = time_edge
    X.index = demand_data.index
    if is_train:
        y = demand_data[
            pd.date_range(date_now, periods=28)
        ].values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X
    


In [31]:
if len(full_price_data.columns[0])==2:
    full_price_data.columns = full_price_data.columns.get_level_values(1)
    full_demand_data.columns = full_demand_data.columns.get_level_values(1)

In [32]:
full_demand_data = full_demand_data.astype(np.float32)
full_price_data = full_price_data.astype(np.float32)

In [33]:
np.unique(calendar_df['event_type_1'].values)

array([0, 1, 2, 3, 4])

In [34]:
calendar_df

,date,wm_yr_wk,wday,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,1,d_1,19,2,3,1,0,0,0
1,2011-01-30,11101,2,d_2,19,2,3,1,0,0,0
2,2011-01-31,11101,3,d_3,19,2,3,1,0,0,0
3,2011-02-01,11101,4,d_4,19,2,3,1,1,1,0
4,2011-02-02,11101,5,d_5,19,2,3,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,5,d_1965,19,2,3,1,0,1,1
1965,2016-06-16,11620,6,d_1966,19,2,3,1,0,0,0
1966,2016-06-17,11620,7,d_1967,19,2,3,1,0,0,0
1967,2016-06-18,11621,1,d_1968,19,2,3,1,0,0,0


In [35]:
time_val = 1913 - 28
time_test = 1913
times_train = [1913 - 28 - 28 - i for i in range(100)]
# product = sales_train_validation_df[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

# def get_calendar_feature(time_edge):
#     cal_features = ['event_type_1', 'event_name_2', 'event_type_2', 'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI']
#     for cal_f in cal_features:
#         for i in 
#         X[cal_f] = calendar_data[cal_f].values[time_edge:]

def get_dataset_for_time(time_edge, test=False):
    if test:
        X_full = prepare_dataset(full_demand_data, full_price_data, time_edge, calendar_df, is_train=False)
    else:
        X_full, y = prepare_dataset(full_demand_data, full_price_data, time_edge, calendar_df, is_train=True)

    
    if test:
        return X_full
    else:
        return X_full, y

print("Preparing dataset...")
Xs_train, ys_train = [], []
for time_train in times_train:
    
    X, y = get_dataset_for_time(time_train)
    
    Xs_train.append(X)
    ys_train.append(y)

X_train = pd.concat(Xs_train, axis=0)
y_train = np.concatenate(ys_train, axis=0)

X_val, y_val = get_dataset_for_time(time_val)

X_test = get_dataset_for_time(time_test, test=True)

Preparing dataset...
2016-02-29
2016-02-28
2016-02-27
2016-02-26
2016-02-25
2016-02-24
2016-02-23
2016-02-22
2016-02-21
2016-02-20
2016-02-19
2016-02-18
2016-02-17
2016-02-16
2016-02-15
2016-02-14
2016-02-13
2016-02-12
2016-02-11
2016-02-10
2016-02-09
2016-02-08
2016-02-07
2016-02-06
2016-02-05
2016-02-04
2016-02-03
2016-02-02
2016-02-01
2016-01-31
2016-01-30
2016-01-29
2016-01-28
2016-01-27
2016-01-26
2016-01-25
2016-01-24
2016-01-23
2016-01-22
2016-01-21
2016-01-20
2016-01-19
2016-01-18
2016-01-17
2016-01-16
2016-01-15
2016-01-14
2016-01-13
2016-01-12
2016-01-11
2016-01-10
2016-01-09
2016-01-08
2016-01-07
2016-01-06
2016-01-05
2016-01-04
2016-01-03
2016-01-02
2016-01-01
2015-12-31
2015-12-30
2015-12-29
2015-12-28
2015-12-27
2015-12-26
2015-12-25
2015-12-24
2015-12-23
2015-12-22
2015-12-21
2015-12-20
2015-12-19
2015-12-18
2015-12-17
2015-12-16
2015-12-15
2015-12-14
2015-12-13
2015-12-12
2015-12-11
2015-12-10
2015-12-09
2015-12-08
2015-12-07
2015-12-06
2015-12-05
2015-12-04
2015-12-03


In [36]:
print(X_train.shape, y_train.shape)

(3049000, 121) (3049000, 28)


In [37]:
X_train

,,,,,,price_after_week_0,price_after_week_1,price_after_week_2,price_after_week_3,price_after_week_4,price_after_week_5,price_after_week_6,price_after_week_7,price_diff_after_week_1,price_diff_after_week_2,price_diff_after_week_3,price_diff_after_week_4,price_diff_after_week_5,price_diff_after_week_6,price_diff_after_week_7,demand_diff_3_mean,demand_mean_3_decay,demand_mean_3,demand_median_3,demand_min_3,demand_max_3,demand_std_3,demand_diff_7_mean,demand_mean_7_decay,demand_mean_7,demand_median_7,demand_min_7,demand_max_7,demand_std_7,demand_diff_14_mean,demand_mean_14_decay,demand_mean_14,demand_median_14,demand_min_14,demand_max_14,demand_std_14,demand_diff_30_mean,demand_mean_30_decay,demand_mean_30,demand_median_30,demand_min_30,demand_max_30,demand_std_30,demand_diff_60_mean,demand_mean_60_decay,demand_mean_60,demand_median_60,demand_min_60,demand_max_60,demand_std_60,demand_diff_140_mean,demand_mean_140_decay,demand_mean_140,demand_median_140,demand_min_140,demand_max_140,demand_std_140,demand_thiswd_diff_3_mean,demand_thiswd_mean_3_decay,demand_thiswd_mean_3,demand_thiswd_median_3,demand_thiswd_min_3,demand_thiswd_max_3,demand_thiswd_std_3,demand_thiswd_diff_7_mean,demand_thiswd_mean_7_decay,demand_thiswd_mean_7,demand_thiswd_median_7,demand_thiswd_min_7,demand_thiswd_max_7,demand_thiswd_std_7,demand_thiswd_diff_14_mean,demand_thiswd_mean_14_decay,demand_thiswd_mean_14,demand_thiswd_median_14,demand_thiswd_min_14,demand_thiswd_max_14,demand_thiswd_std_14,price_diff_4_mean,price_mean_4_decay,price_mean_4,price_min_4,price_max_4,price_std_4,price_diff_10_mean,price_mean_10_decay,price_mean_10,price_min_10,price_max_10,price_std_10,weights,day_shift_1,day_shift_2,day_shift_3,day_shift_4,day_shift_5,day_shift_6,day_shift_7,day_shift_8,day_shift_9,day_shift_10,day_shift_11,day_shift_12,day_shift_13,day_shift_14,day_shift_15,day_diff_shift_1,day_diff_shift_2,day_diff_shift_3,day_diff_shift_4,day_diff_shift_5,day_diff_shift_6,day_diff_shift_7,day_diff_shift_8,day_diff_shift_9,day_diff_shift_10,day_diff_shift_11,day_diff_shift_12,day_diff_shift_13,day_diff_shift_14,time_edge
id,item_id,dept_id,cat_id,store_id,state_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.590490,0.142857,0.0,0.0,1.0,0.377964,-0.076923,2.406842,0.428571,0.0,0.0,2.0,0.646206,0.000000,4.199849,0.733333,0.0,0.0,3.0,0.944433,-0.033898,4.450667,0.616667,0.0,0.0,3.0,0.884742,-0.007194,4.459765,0.664286,0.0,0.0,6.0,0.986314,-1.0,2.52,1.000000,1.0,0.0,2.0,1.000000,0.000000,2.520000,0.428571,0.0,0.0,2.0,0.786796,-0.076923,3.880105,0.500000,0.0,0.0,2.0,0.759555,0.0,7.704165,2.240234,2.240234,2.240234,0.0,0.0,14.591129,2.240234,2.240234,2.240234,0.0,230.744141,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,-2.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1857
FOODS_1_001_CA_2_validation,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,-0.500000,2.775303,0.714286,0.0,0.0,3.0,1.253566,-0.230769,8.378218,1.428571,1.0,0.0,6.0,1.741542,-0.034483,10.473420,1.233333,1.0,0.0,6.0,1.524135,-0.067797,10.875628,1.133333,1.0,0.0,6.0,1.599435,-0.007194,10.893002,1.064286,0.0,0.0,11.0,1.680230,1.0,6.51,2.333333,3.0,1.0,3.0,1.154701,0.500000,8.624100,1.428571,1.0,0.0,3.0,1.272418,0.230769,9.102396,0.785714,0.0,0.0,3.0,1.121714,0.0,7.704165,2.240234,2.240234,2.240234,0.0,0.0,14.591129,2.240234,2.240234,2.240234,0.0,371.878906,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,6.0,1.0,1.0,0.0,2.0,3.0,3.0,0.0,0.0,0.0,0.0,-2.0,-1.0,1.0,-4.0,5.0,0.0,1.0,-2.0,-1.0,0.0,1857
FOODS_1_001_CA_3_validation,0,0,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.309999,3.666667,5.0,1.0,5.0,2.309401,0.833333,11.039000,1.714286,1.0,0.0,5.0,2.288688,0.384615,15.292973,1.642857,1.0,0.

In [38]:
X_train = X_train.reset_index(level=[1, 2, 3, 4, 5])
X_val = X_val.reset_index(level=[1, 2, 3, 4, 5])
X_test = X_test.reset_index(level=[1, 2, 3, 4, 5])

In [39]:
X_train

,item_id,dept_id,cat_id,store_id,state_id,price_after_week_0,price_after_week_1,price_after_week_2,price_after_week_3,price_after_week_4,price_after_week_5,price_after_week_6,price_after_week_7,price_diff_after_week_1,price_diff_after_week_2,price_diff_after_week_3,price_diff_after_week_4,price_diff_after_week_5,price_diff_after_week_6,price_diff_after_week_7,demand_diff_3_mean,demand_mean_3_decay,demand_mean_3,demand_median_3,demand_min_3,demand_max_3,demand_std_3,demand_diff_7_mean,demand_mean_7_decay,demand_mean_7,demand_median_7,demand_min_7,demand_max_7,demand_std_7,demand_diff_14_mean,demand_mean_14_decay,demand_mean_14,demand_median_14,demand_min_14,demand_max_14,demand_std_14,demand_diff_30_mean,demand_mean_30_decay,demand_mean_30,demand_median_30,demand_min_30,demand_max_30,demand_std_30,demand_diff_60_mean,demand_mean_60_decay,demand_mean_60,demand_median_60,demand_min_60,demand_max_60,demand_std_60,demand_diff_140_mean,demand_mean_140_decay,demand_mean_140,demand_median_140,demand_min_140,demand_max_140,demand_std_140,demand_thiswd_diff_3_mean,demand_thiswd_mean_3_decay,demand_thiswd_mean_3,demand_thiswd_median_3,demand_thiswd_min_3,demand_thiswd_max_3,demand_thiswd_std_3,demand_thiswd_diff_7_mean,demand_thiswd_mean_7_decay,demand_thiswd_mean_7,demand_thiswd_median_7,demand_thiswd_min_7,demand_thiswd_max_7,demand_thiswd_std_7,demand_thiswd_diff_14_mean,demand_thiswd_mean_14_decay,demand_thiswd_mean_14,demand_thiswd_median_14,demand_thiswd_min_14,demand_thiswd_max_14,demand_thiswd_std_14,price_diff_4_mean,price_mean_4_decay,price_mean_4,price_min_4,price_max_4,price_std_4,price_diff_10_mean,price_mean_10_decay,price_mean_10,price_min_10,price_max_10,price_std_10,weights,day_shift_1,day_shift_2,day_shift_3,day_shift_4,day_shift_5,day_shift_6,day_shift_7,day_shift_8,day_shift_9,day_shift_10,day_shift_11,day_shift_12,day_shift_13,day_shift_14,day_shift_15,day_diff_shift_1,day_diff_shift_2,day_diff_shift_3,day_diff_shift_4,day_diff_shift_5,day_diff_shift_6,day_diff_shift_7,day_diff_shift_8,day_diff_shift_9,day_diff_shift_10,day_diff_shift_11,day_diff_shift_12,day_diff_shift_13,day_diff_shift_14,time_edge
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.590490,0.142857,0.0,0.0,1.0,0.377964,-0.076923,2.406842,0.428571,0.0,0.0,2.0,0.646206,0.000000,4.199849,0.733333,0.0,0.0,3.0,0.944433,-0.033898,4.450667,0.616667,0.0,0.0,3.0,0.884742,-0.007194,4.459765,0.664286,0.0,0.0,6.0,0.986314,-1.0,2.52,1.000000,1.0,0.0,2.0,1.000000,0.000000,2.520000,0.428571,0.0,0.0,2.0,0.786796,-0.076923,3.880105,0.500000,0.0,0.0,2.0,0.759555,0.0,7.704165,2.240234,2.240234,2.240234,0.0,0.0,14.591129,2.240234,2.240234,2.240234,0.0,230.744141,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,-2.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1857
FOODS_1_001_CA_2_validation,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,-0.500000,2.775303,0.714286,0.0,0.0,3.0,1.253566,-0.230769,8.378218,1.428571,1.0,0.0,6.0,1.741542,-0.034483,10.473420,1.233333,1.0,0.0,6.0,1.524135,-0.067797,10.875628,1.133333,1.0,0.0,6.0,1.599435,-0.007194,10.893002,1.064286,0.0,0.0,11.0,1.680230,1.0,6.51,2.333333,3.0,1.0,3.0,1.154701,0.500000,8.624100,1.428571,1.0,0.0,3.0,1.272418,0.230769,9.102396,0.785714,0.0,0.0,3.0,1.121714,0.0,7.704165,2.240234,2.240234,2.240234,0.0,0.0,14.591129,2.240234,2.240234,2.240234,0.0,371.878906,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,6.0,1.0,1.0,0.0,2.0,3.0,3.0,0.0,0.0,0.0,0.0,-2.0,-1.0,1.0,-4.0,5.0,0.0,1.0,-2.0,-1.0,0.0,1857
FOODS_1_001_CA_3_validation,0,0,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.309999,3.666667,5.0,1.0,5.0,2.309401,0.833333,11.039000,1.714286,1.0,0.0,5.0,2.288688,0.384615,15.292973,1.642857,1.0,0.

In [40]:
X_test

,item_id,dept_id,cat_id,store_id,state_id,price_after_week_0,price_after_week_1,price_after_week_2,price_after_week_3,price_after_week_4,price_after_week_5,price_after_week_6,price_after_week_7,price_diff_after_week_1,price_diff_after_week_2,price_diff_after_week_3,price_diff_after_week_4,price_diff_after_week_5,price_diff_after_week_6,price_diff_after_week_7,demand_diff_3_mean,demand_mean_3_decay,demand_mean_3,demand_median_3,demand_min_3,demand_max_3,demand_std_3,demand_diff_7_mean,demand_mean_7_decay,demand_mean_7,demand_median_7,demand_min_7,demand_max_7,demand_std_7,demand_diff_14_mean,demand_mean_14_decay,demand_mean_14,demand_median_14,demand_min_14,demand_max_14,demand_std_14,demand_diff_30_mean,demand_mean_30_decay,demand_mean_30,demand_median_30,demand_min_30,demand_max_30,demand_std_30,demand_diff_60_mean,demand_mean_60_decay,demand_mean_60,demand_median_60,demand_min_60,demand_max_60,demand_std_60,demand_diff_140_mean,demand_mean_140_decay,demand_mean_140,demand_median_140,demand_min_140,demand_max_140,demand_std_140,demand_thiswd_diff_3_mean,demand_thiswd_mean_3_decay,demand_thiswd_mean_3,demand_thiswd_median_3,demand_thiswd_min_3,demand_thiswd_max_3,demand_thiswd_std_3,demand_thiswd_diff_7_mean,demand_thiswd_mean_7_decay,demand_thiswd_mean_7,demand_thiswd_median_7,demand_thiswd_min_7,demand_thiswd_max_7,demand_thiswd_std_7,demand_thiswd_diff_14_mean,demand_thiswd_mean_14_decay,demand_thiswd_mean_14,demand_thiswd_median_14,demand_thiswd_min_14,demand_thiswd_max_14,demand_thiswd_std_14,price_diff_4_mean,price_mean_4_decay,price_mean_4,price_min_4,price_max_4,price_std_4,price_diff_10_mean,price_mean_10_decay,price_mean_10,price_min_10,price_max_10,price_std_10,weights,day_shift_1,day_shift_2,day_shift_3,day_shift_4,day_shift_5,day_shift_6,day_shift_7,day_shift_8,day_shift_9,day_shift_10,day_shift_11,day_shift_12,day_shift_13,day_shift_14,day_shift_15,day_diff_shift_1,day_diff_shift_2,day_diff_shift_3,day_diff_shift_4,day_diff_shift_5,day_diff_shift_6,day_diff_shift_7,day_diff_shift_8,day_diff_shift_9,day_diff_shift_10,day_diff_shift_11,day_diff_shift_12,day_diff_shift_13,day_diff_shift_14,time_edge
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,-0.5,1.71,0.666667,1.0,0.0,1.0,0.577350,-0.666667,5.082354,1.142857,1.0,0.0,4.0,1.345186,-0.153846,7.114150,1.000000,1.0,0.0,4.0,1.109400,-0.034483,9.482983,1.133333,1.0,0.0,4.0,1.252125,0.000000,9.905935,0.950000,1.0,0.0,4.0,1.095832,-0.014388,9.918148,0.814286,0.0,0.0,4.0,1.042864,0.0,9.04,3.333333,4.0,2.0,4.0,1.154701,0.666667,12.269470,2.142857,2.0,0.0,4.0,1.676163,0.307692,13.832544,1.357143,1.0,0.0,4.0,1.499084,0.0,7.704165,2.240234,2.240234,2.240234,0.0,0.0,14.591129,2.240234,2.240234,2.240234,0.0,268.828125,0.0,1.0,1.0,0.0,1.0,1.0,4.0,0.0,2.0,0.0,1.0,1.0,0.0,2.0,2.0,-1.0,0.0,1.0,-1.0,0.0,-3.0,4.0,-2.0,2.0,-1.0,0.0,1.0,-2.0,0.0,1913
FOODS_1_001_CA_2_validation,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0,4.00,1.333333,0.0,0.0,4.0,2.309401,0.666667,8.162590,1.428571,1.0,0.0,4.0,1.812654,0.307692,15.873789,1.928571,0.5,0.0,14.0,3.751190,0.068966,16.577177,1.133333,0.0,0.0,14.0,2.674700,0.067797,16.835567,0.983333,0.0,0.0,14.0,2.119255,0.028777,16.865398,1.078571,0.0,0.0,14.0,1.779451,-0.5,0.81,0.333333,0.0,0.0,1.0,0.577350,-0.333333,4.890942,1.142857,1.0,0.0,4.0,1.463850,0.000000,8.603334,1.285714,1.0,0.0,4.0,1.325987,0.0,7.704165,2.240234,2.240234,2.240234,0.0,0.0,14.591129,2.240234,2.240234,2.240234,0.0,369.638672,4.0,0.0,0.0,4.0,1.0,1.0,0.0,14.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,-4.0,3.0,0.0,1.0,-14.0,14.0,-1.0,1.0,-2.0,2.0,0.0,0.0,1913
FOODS_1_001_CA_3_validation,0,0,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.90,0.333333,0.0,0.0,1.0,0.577350,0.000000,0.900000,0.142857,0.0,0.0,1.0,0.3779

In [41]:
def get_calendar_features(calendar_data, day):
    cal_features = ['event_type_1', 'event_name_2', 'event_type_2', 'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI', 'wday']
    X = {}
    X = pd.DataFrame(X)
    for cal_f in cal_features:
        for i in np.arange(-2, 5):
            #print(day + i)
            X[f'{cal_f}_after_{i}'] = calendar_data[cal_f].values[day + i]
            X[f'{cal_f}_after_{i}'] = X[f'{cal_f}_after_{i}'].astype('category')
    
#     date_now = str(calendar_data['date'].values[day]).split('T')[0]
#     print(date_now)
#     year, month, day = np.array(date_now.split('-')).astype(np.int)
#     date_now = date(year, month, day)
#     X['dayofweek'] = date_now.weekday()#.astype('category')
#     X["is_weekend"] = (date_now.weekday() in [5,6])
    return X

In [45]:
params = {
    'num_leaves': 80,
    'objective': 'poisson',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'rmse',
    'n_jobs': -1
}

MAX_ROUNDS = 2000
val_pred = []
test_pred = []
cate_vars = []

def add_cal_features(X, step):
    time_edge = X['time_edge'].values
    cal_fs = get_calendar_features(calendar_df, time_edge + step)
    cal_fs.index = X.index
    weights = X['weights'].values
    weights /= np.sum(weights)
    X = X.drop(['weights'], axis=1)
    return pd.concat([X, cal_fs], axis=1), weights

for i in range(28):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    
    X_train_i, weights_train = add_cal_features(X_train, i)
    X_val_i, weights_val = add_cal_features(X_val, i)
    X_test_i, _ = add_cal_features(X_test, i)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i]#, weight=weights_train
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i]#, weight=weights_val)
    )
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
    )
#     print("\n".join(("%s: %.2f" % x) for x in sorted(
#         zip(X_train.columns, bst.feature_importance("gain")),
#         key=lambda x: x[1], reverse=True
#     )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

    joblib.dump(bst, 'lgbm_0.sav')

Step 1
Training until validation scores don't improve for 125 rounds
[50]	training's rmse: 2.66371	valid_1's rmse: 2.10515
[100]	training's rmse: 2.27285	valid_1's rmse: 1.82647
[150]	training's rmse: 2.09927	valid_1's rmse: 1.77857
[200]	training's rmse: 2.01999	valid_1's rmse: 1.79764
[250]	training's rmse: 1.97755	valid_1's rmse: 1.81504
Early stopping, best iteration is:
[147]	training's rmse: 2.10586	valid_1's rmse: 1.7784
Step 2
Training until validation scores don't improve for 125 rounds
[50]	training's rmse: 2.69484	valid_1's rmse: 2.20863
[100]	training's rmse: 2.32384	valid_1's rmse: 1.93428
[150]	training's rmse: 2.1575	valid_1's rmse: 1.8657
[200]	training's rmse: 2.0775	valid_1's rmse: 1.86157
[250]	training's rmse: 2.03248	valid_1's rmse: 1.86865
[300]	training's rmse: 2.00425	valid_1's rmse: 1.87416
Early stopping, best iteration is:
[183]	training's rmse: 2.09778	valid_1's rmse: 1.86067
Step 3
Training until validation scores don't improve for 125 rounds
[50]	training'

[350]	training's rmse: 2.05169	valid_1's rmse: 2.19537
[400]	training's rmse: 2.03475	valid_1's rmse: 2.19359
[450]	training's rmse: 2.01942	valid_1's rmse: 2.19323
[500]	training's rmse: 2.00684	valid_1's rmse: 2.19426
Early stopping, best iteration is:
[408]	training's rmse: 2.03233	valid_1's rmse: 2.19289
Step 13
Training until validation scores don't improve for 125 rounds
[50]	training's rmse: 2.76368	valid_1's rmse: 3.69037
[100]	training's rmse: 2.40193	valid_1's rmse: 3.21465
[150]	training's rmse: 2.23514	valid_1's rmse: 2.98142
[200]	training's rmse: 2.15429	valid_1's rmse: 2.85839
[250]	training's rmse: 2.11055	valid_1's rmse: 2.78461
[300]	training's rmse: 2.0817	valid_1's rmse: 2.74025
[350]	training's rmse: 2.05754	valid_1's rmse: 2.71185
[400]	training's rmse: 2.04018	valid_1's rmse: 2.6947
[450]	training's rmse: 2.02362	valid_1's rmse: 2.68024
[500]	training's rmse: 2.01066	valid_1's rmse: 2.6712
[550]	training's rmse: 1.99848	valid_1's rmse: 2.66332
[600]	training's rm

[850]	training's rmse: 1.96273	valid_1's rmse: 2.53461
[900]	training's rmse: 1.95648	valid_1's rmse: 2.53436
[950]	training's rmse: 1.95064	valid_1's rmse: 2.53532
[1000]	training's rmse: 1.94486	valid_1's rmse: 2.53594
Early stopping, best iteration is:
[882]	training's rmse: 1.9589	valid_1's rmse: 2.53344
Step 22
Training until validation scores don't improve for 125 rounds
[50]	training's rmse: 2.75709	valid_1's rmse: 2.53922
[100]	training's rmse: 2.39226	valid_1's rmse: 2.24243
[150]	training's rmse: 2.22814	valid_1's rmse: 2.14051
[200]	training's rmse: 2.1508	valid_1's rmse: 2.11322
[250]	training's rmse: 2.10925	valid_1's rmse: 2.10762
[300]	training's rmse: 2.08165	valid_1's rmse: 2.10805
[350]	training's rmse: 2.06341	valid_1's rmse: 2.10871
Early stopping, best iteration is:
[263]	training's rmse: 2.10054	valid_1's rmse: 2.10721
Step 23
Training until validation scores don't improve for 125 rounds
[50]	training's rmse: 2.75979	valid_1's rmse: 2.37442
[100]	training's rmse: 

In [46]:
val_pred = np.array(val_pred)
test_pred = np.array(test_pred)

In [47]:
val_pred = np.transpose(val_pred)
test_pred = np.transpose(test_pred)

In [48]:
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score is {val_score}')
prediction_df = pd.DataFrame(data = test_pred, index = X_test.index.get_level_values(0))
prediction_df = prediction_df.reindex(submission_df['id']).fillna(0)
prediction_df

Our val rmse score is 2.14869172083683


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,0.984295,0.998067,1.025404,1.023291,0.947858,1.088639,1.203826,0.944768,0.953862,0.944574,1.005425,0.935841,1.106419,1.058619,0.921068,0.997270,1.037239,0.972223,0.927892,1.068768,1.019358,0.902325,0.921872,1.048642,1.206292,0.970061,1.030226,1.004702
HOBBIES_1_002_CA_1_validation,0.467844,0.387480,0.418774,0.357809,0.263949,0.271173,0.302076,0.336121,0.320081,0.280197,0.291443,0.282669,0.302412,0.301328,0.319124,0.379460,0.422680,0.281425,0.275501,0.243796,0.265701,0.327140,0.297654,0.236145,0.239361,0.319627,0.332108,0.427529
HOBBIES_1_003_CA_1_validation,0.809096,0.771899,0.782871,0.743953,0.627430,0.671913,0.772310,0.665548,0.671346,0.603563,0.678186,0.640422,0.681940,0.731089,0.632659,0.710825,0.783789,0.663015,0.625596,0.655408,0.649769,0.653789,0.643157,0.508435,0.483881,0.650948,0.672551,0.763310
HOBBIES_1_004_CA_1_validation,2.013160,1.989459,1.890154,1.934647,1.997843,2.500683,2.855138,2.176601,1.960523,1.881778,1.921421,1.953303,2.440851,2.510403,2.045639,1.912204,1.902816,1.779000,1.842758,2.231152,2.430297,2.031903,1.822955,1.690437,1.621544,2.011700,2.134352,2.019923
HOBBIES_1_005_CA_1_validation,1.261839,1.268893,1.277788,1.319989,1.306500,1.435992,1.605419,1.242437,1.227106,1.207958,1.285444,1.307956,1.458403,1.444853,1.214110,1.222363,1.236146,1.176307,1.242514,1.330369,1.364510,1.222573,1.197892,1.288816,1.202663,1.303900,1.308463,1.314460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
submission_df.iloc[:, 1:] = prediction_df.values

submission_df.to_csv('submission_my_lgbm_thatparams.csv', index = False)

### Help taken from these kernels

1. https://www.kaggle.com/ragnar123/very-fst-model



> Note: If you like my work, please, upvote ☺

In [ ]:
# submission_df = pd.read_csv('submission_my_lgbm.csv')
# submission_df

In [ ]:
# submission_df = pd.read_csv('submission.csv')
# submission_df

In [ ]:
# np.mean(prediction_df.values/(submission_df.values[:, 1:] + 1e-3), axis=1)[:20]

In [ ]:
!kaggle competitions submit -c m5-forecasting-accuracy -f submission_my_lgbm_thatparams.csv -m "Fixed categories"